# Notebook for scraping Korea

I created a separate notebook to scrape different regions. This allows me to collect games faster.

Load libraries

In [1]:
import requests, json
import numpy as np
import API_io.API_io as API_io
import importlib
import pandas as pd

In [2]:
working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\lolML'
with open(working_dir+ '\\api_key.txt', 'r') as api_file:
    api_key =  api_file.read()

## Get a list of high level matches

Get summoners in featured games

In [11]:
featured_json = API_io.load_featured_games(api_key, 'kr') # load json of featured games
featured_game_ids = [x['gameId'] for x in featured_json ] #  use list comprehension to get featured games; don't use this

Make a list of summoner names and summoner IDs from the featured JSON

In [23]:
summoner_names, summoner_IDs = API_io.get_summoners_IDs_from_featured_games(featured_json, api_key, region ='kr')
summoner_names[:5]

['하얀 눈길', '비제이대복이', '빡겜가자', '병기형이다z', '모래반지']

In [28]:
summoner_urls = [API_io.make_matchlist_url_summoner_ID(x, True, True, api_key, region ='kr') for x in summoner_IDs]
print(summoner_urls[:2])
match_histories = [API_io.get_limited_request(x) for x in summoner_urls ]

Extract the match ID from the match history JSON

In [45]:
match_IDs = np.empty(0, dtype=int)
for cur_matches in match_histories:
    match_IDs = np.append( match_IDs, [x['matchId'] for x in cur_matches['matches']] )
match_IDs = np.unique(match_IDs)
pd.Series(match_IDs).to_csv('Korean Match IDs.csv')
match_IDs.shape

(32754,)

In [39]:
# make urls for loading
match_urls = [API_io.make_match_info_url(x, True, api_key, region = 'kr') for x in match_IDs] # True flag means we get the timeline
match_urls[:2]

['https://kr.api.pvp.net/api/lol/kr/v2.2/match/1713275489?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7',
 'https://kr.api.pvp.net/api/lol/kr/v2.2/match/1713841964?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7']

In [41]:
import time
from API_io import feature_calc

In [68]:
match_range = np.arange(1000, 1100)
col_names = feature_calc.col_names 
analysis_min = 20 # analyze at 20 minutes
games_df = pd.DataFrame(index = match_range, columns= col_names)
# this for loop is ugly; used list comprehension previously, but rate limit was fluky
for cur_match in match_range:
    time.sleep(1.2) # RIOT API is throttled to ~0.83 requests / second
    try:
        match_info = requests.get(match_urls[cur_match]).json()
        games_df.loc[cur_match] = feature_calc.calc_features_single_match(match_info, analysis_min)
    except requests.exceptions.HTTPError as err:
        print('HHTPError in game ' + str(match_IDs[cur_match]))
    except:
        import sys
        err = sys.exc_info()[0]
        print('Error: ' + str(err) + ' in game ' + str(match_IDs[cur_match]))
        
games_df = games_df.dropna()
games_df = feature_calc.retype_columns(games_df)   

Error: <class 'AssertionError'> in game 1728156495
Error: <class 'AssertionError'> in game 1728206733
Error: <class 'KeyError'> in game 1728218289
Error: <class 'AssertionError'> in game 1728241529
Error: <class 'AssertionError'> in game 1728243255
Error: <class 'KeyError'> in game 1728263628
Error: <class 'ValueError'> in game 1728953264
Error: <class 'AssertionError'> in game 1728959374
Error: <class 'AssertionError'> in game 1729111707
Error: <class 'AssertionError'> in game 1729145588


Now save the information to disk.

In [59]:
games_df.to_csv('Korean games 1-1000.csv')